<a href="https://colab.research.google.com/github/naimadmpi10/video-subtitle-/blob/main/ArabicToBanglaSubtitle8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Step 2: প্রয়োজনীয় লাইব্রেরি ইনস্টল করুন
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q deep-translator

In [ ]:

# Step 3: ভিডিও ফাইলের পথ দিন (আপনার ভিডিও Drive-এ আপলোড করা থাকতে হবে)
video_path = "/content/drive/MyDrive/Videos/as.mp4"  # এখানে ফাইলের নাম ও path দিন
!apt-get update && apt-get install -y ffmpeg

In [ ]:

# Step 4: ভিডিও থেকে ট্রান্সক্রিপশন করুন
import whisper

model = whisper.load_model("base")
result = model.transcribe(video_path)
segments = result['segments']

In [ ]:

from deep_translator import GoogleTranslator

translated_segments = []
for seg in result['segments']:
    bn_text = GoogleTranslator(source='ar', target='bn').translate(seg['text'])
    translated_segments.append((seg['start'], seg['end'], bn_text))

In [ ]:
!apt install fonts-noto

In [ ]:
# Step 5: .ASS সাবটাইটেল ফাইল তৈরি করা
def format_ass_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 100)
    return f"{hours}:{minutes:02}:{secs:02}.{millis:02}"

ass_header = """
[Script Info]
Title: Bengali Subtitles
ScriptType: v4.00+
PlayResY: 720
PlayResX: 1280
Timer: 100.0000

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, BackColour, OutlineColour, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,Noto Sans Bengali,36,&H00FFFFFF,&H00000000,&H00000000,1,2,0,2,10,10,30,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

ass_path = "/content/drive/MyDrive/subtitles/Oas.ass"
with open(ass_path, "w", encoding="utf-8") as f:
    f.write(ass_header)
    for start, end, text in translated_segments:
        f.write(f"Dialogue: 0,{format_ass_time(start)},{format_ass_time(end)},Default,,0,0,0,,{text}\n")

In [ ]:
# Step 6: ভিডিওতে বাংলা সাবটাইটেল বার্ন করা
!ffmpeg -i "/content/drive/MyDrive/Videos/as.mp4" -vf "ass=/content/drive/MyDrive/subtitles/Oas.ass" -c:a copy "/content/drive/MyDrive/Videos/output.mp4"